In [ ]:
from capsnet.fine_tuning_helper import load_study

study_name = "main"
study = load_study(study_name)

In [ ]:
try:
    display(study.best_value)
    display(study.best_params)
except:
    print("Empty Study")

Empty Study


In [ ]:
from capsnet.dataset_helper import load_raw_datasets

train_dataset_raw, val_dataset_raw, test_dataset_raw = load_raw_datasets()

In [ ]:
from capsnet.fine_tuning_helper import objective, save_study
from capsnet.dataset_helper import CacheLoader
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cache_loader = CacheLoader(train_dataset_raw, val_dataset_raw)

study.optimize(
    lambda trial: objective(trial, 15, device, cache_loader),
    n_trials=10
)

save_study(study_name, study)

c:\Users\RINGULARITY\venvs\model_train\Lib\site-packages\albumentations\core\validation.py:58: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
100%|██████████| 72/72 [00:00<00:00, 1763.32it/s]
c:\Users\RINGULARITY\venvs\model_train\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2025-03-04 20:01:04,242] Trial 0 finished with value: 0.0 and parameters: {'input_size': 32, 'lr': 0.008685180967642222, 'alpha': 0.000339062627723243, 'routing_iters': 3, 'num_capsules': 58, 'capsule_dim': 5, 'primary_kernel_size': 3, 'primary_stride': 2, 'n_decoder_layers': 2, 'decoder_hidden_dim_0': 512, 'decoder_dropout_rate_0': 0.29730769834564896, 'decoder_hidden_dim_1': 128, 'decoder_dropout_rate_1': 0.4360000651975947, 'decoder_final_dropout': 0.3282070276692439}. Best is trial 0 w

Trial 0 finished with value: 0.0 and parameters: {'input_size': 32, 'lr': 0.008685180967642222, 'alpha': 0.000339062627723243, 'routing_iters': 3, 'num_capsules': 58, 'capsule_dim': 5, 'primary_kernel_size': 3, 'primary_stride': 2, 'n_decoder_layers': 2, 'decoder_hidden_dim_0': 512, 'decoder_dropout_rate_0': 0.29730769834564896, 'decoder_hidden_dim_1': 128, 'decoder_dropout_rate_1': 0.4360000651975947, 'decoder_final_dropout': 0.3282070276692439}. Best is trial 0 with value: 0.0.


[I 2025-03-04 20:01:04,372] Trial 1 finished with value: 0.0 and parameters: {'input_size': 32, 'lr': 0.001084242268707629, 'alpha': 0.0029123752142354276, 'routing_iters': 2, 'num_capsules': 58, 'capsule_dim': 5, 'primary_kernel_size': 8, 'primary_stride': 2, 'n_decoder_layers': 2, 'decoder_hidden_dim_0': 128, 'decoder_dropout_rate_0': 0.22941637535623144, 'decoder_hidden_dim_1': 4096, 'decoder_dropout_rate_1': 0.3195730688305695, 'decoder_final_dropout': 0.37404193793413426}. Best is trial 0 with value: 0.0.


Trial 1 finished with value: 0.0 and parameters: {'input_size': 32, 'lr': 0.001084242268707629, 'alpha': 0.0029123752142354276, 'routing_iters': 2, 'num_capsules': 58, 'capsule_dim': 5, 'primary_kernel_size': 8, 'primary_stride': 2, 'n_decoder_layers': 2, 'decoder_hidden_dim_0': 128, 'decoder_dropout_rate_0': 0.22941637535623144, 'decoder_hidden_dim_1': 4096, 'decoder_dropout_rate_1': 0.3195730688305695, 'decoder_final_dropout': 0.37404193793413426}. Best is trial 0 with value: 0.0.


c:\Users\RINGULARITY\venvs\model_train\Lib\site-packages\albumentations\core\validation.py:58: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
100%|██████████| 72/72 [00:00<00:00, 1416.83it/s]
c:\Users\RINGULARITY\venvs\model_train\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2025-03-04 20:01:25,575] Trial 2 finished with value: 0.0 and parameters: {'input_size': 64, 'lr': 0.009995782111399426, 'alpha': 0.0007118822524635332, 'routing_iters': 3, 'num_capsules': 24, 'capsule_dim': 9, 'primary_kernel_size': 4, 'primary_stride': 3, 'n_decoder_layers': 2, 'decoder_hidden_dim_0': 2048, 'decoder_dropout_rate_0': 0.31595194360710377, 'decoder_hidden_dim_1': 256, 'decoder_dropout_rate_1': 0.22950985922990347, 'decoder_final_dropout': 0.25496546213719684}. Best is trial

...

Trial 7 finished with value: 0.0 and parameters: {'input_size': 64, 'lr': 0.006273768784604809, 'alpha': 0.0002970642110946409, 'routing_iters': 2, 'num_capsules': 27, 'capsule_dim': 16, 'primary_kernel_size': 4, 'primary_stride': 3, 'n_decoder_layers': 1, 'decoder_hidden_dim_0': 256, 'decoder_dropout_rate_0': 0.2803327175923568, 'decoder_final_dropout': 0.3100296194726768}. Best is trial 4 with value: 0.9953623188405797.


c:\Users\RINGULARITY\venvs\model_train\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2025-03-04 20:17:26,667] Trial 8 finished with value: 0.0 and parameters: {'input_size': 128, 'lr': 0.007836578035052584, 'alpha': 0.00013045147892589372, 'routing_iters': 5, 'num_capsules': 56, 'capsule_dim': 8, 'primary_kernel_size': 3, 'primary_stride': 3, 'n_decoder_layers': 1, 'decoder_hidden_dim_0': 256, 'decoder_dropout_rate_0': 0.23047383434502666, 'decoder_final_dropout': 0.2591988255018764}. Best is trial 4 with value: 0.9953623188405797.


Trial 8 finished with value: 0.0 and parameters: {'input_size': 128, 'lr': 0.007836578035052584, 'alpha': 0.00013045147892589372, 'routing_iters': 5, 'num_capsules': 56, 'capsule_dim': 8, 'primary_kernel_size': 3, 'primary_stride': 3, 'n_decoder_layers': 1, 'decoder_hidden_dim_0': 256, 'decoder_dropout_rate_0': 0.23047383434502666, 'decoder_final_dropout': 0.2591988255018764}. Best is trial 4 with value: 0.9953623188405797.


c:\Users\RINGULARITY\venvs\model_train\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2025-03-04 20:18:11,185] Trial 9 finished with value: 0.0 and parameters: {'input_size': 128, 'lr': 0.0029991376323844775, 'alpha': 0.0035478865822656087, 'routing_iters': 2, 'num_capsules': 41, 'capsule_dim': 16, 'primary_kernel_size': 5, 'primary_stride': 3, 'n_decoder_layers': 1, 'decoder_hidden_dim_0': 128, 'decoder_dropout_rate_0': 0.2284029804600178, 'decoder_final_dropout': 0.3128167506164513}. Best is trial 4 with value: 0.9953623188405797.


Trial 9 finished with value: 0.0 and parameters: {'input_size': 128, 'lr': 0.0029991376323844775, 'alpha': 0.0035478865822656087, 'routing_iters': 2, 'num_capsules': 41, 'capsule_dim': 16, 'primary_kernel_size': 5, 'primary_stride': 3, 'n_decoder_layers': 1, 'decoder_hidden_dim_0': 128, 'decoder_dropout_rate_0': 0.2284029804600178, 'decoder_final_dropout': 0.3128167506164513}. Best is trial 4 with value: 0.9953623188405797.


In [ ]:
display(study.best_value)
display(study.best_params)

0.9953623188405797

{'input_size': 128,
 'lr': 0.00015591141510305244,
 'alpha': 0.0005686427193811906,
 'routing_iters': 5,
 'num_capsules': 58,
 'capsule_dim': 5,
 'primary_kernel_size': 4,
 'primary_stride': 2,
 'n_decoder_layers': 2,
 'decoder_hidden_dim_0': 512,
 'decoder_dropout_rate_0': 0.35738198055304526,
 'decoder_hidden_dim_1': 1024,
 'decoder_dropout_rate_1': 0.42925866582391436,
 'decoder_final_dropout': 0.4388723583138437}

In [ ]:
from capsnet.fine_tuning_helper import study_to_model_params
from capsnet.model import CapsNet
from capsnet.train_helper import train_capsnet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_best_params, alpha, lr = study_to_model_params(study.best_params, conv_channels=[64, 128, 256], num_classes=6, capsule_out_dim=16)

train_loader, val_loader = cache_loader.get_or_create_cache(model_best_params["input_size"])

best_model = CapsNet(**model_best_params).to(device)

best_model_trained, top3_f1_avg = train_capsnet(best_model, train_loader, val_loader, device, epochs=20, alpha=alpha, lr=lr, progress=True)

c:\Users\RINGULARITY\venvs\model_train\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20 - Train Loss: 0.4394, Val Loss: 0.2882, Val F1: 66.16%
Epoch 2/20 - Train Loss: 0.3121, Val Loss: 0.2547, Val F1: 72.09%
Epoch 3/20 - Train Loss: 0.2996, Val Loss: 0.1947, Val F1: 83.27%
Epoch 4/20 - Train Loss: 0.2230, Val Loss: 0.1183, Val F1: 93.11%
Epoch 5/20 - Train Loss: 0.2027, Val Loss: 0.0983, Val F1: 93.23%
Epoch 6/20 - Train Loss: 0.1889, Val Loss: 0.0833, Val F1: 91.59%
Epoch 7/20 - Train Loss: 0.1625, Val Loss: 0.0671, Val F1: 93.14%
Epoch 8/20 - Train Loss: 0.1391, Val Loss: 0.0586, Val F1: 97.20%
Epoch 9/20 - Train Loss: 0.1226, Val Loss: 0.0497, Val F1: 94.59%
Epoch 10/20 - Train Loss: 0.1000, Val Loss: 0.0461, Val F1: 98.61%
Epoch 11/20 - Train Loss: 0.0862, Val Loss: 0.0436, Val F1: 98.61%
Epoch 12/20 - Train Loss: 0.0761, Val Loss: 0.0323, Val F1: 98.61%
Epoch 13/20 - Train Loss: 0.0637, Val Loss: 0.0583, Val F1: 90.78%
Epoch 14/20 - Train Loss: 0.0623, Val Loss: 0.0134, Val F1: 100.00%
Epoch 15/20 - Train Loss: 0.0561, Val Loss: 0.0179, Val F1: 100.00%
Ep

In [ ]:
from capsnet.model import save_model

save_model(best_model_trained, model_best_params, "main")